In [2]:

from __future__ import annotations
from datetime import datetime, timezone
from syft_event import SyftEvents

from syft_core import Client
from syft_rds.models.dataset import Dataset, DatasetCreate
from syft_rds.service.dataset_service import DatasetService
from syft_rds.connection.connection import get_connection
from syft_rds.service.context import BaseRPCContext

In [3]:
client = Client.load()

In [4]:

from syft_core import SyftBoxURL


box = SyftEvents("my-rds-app")

# TODO: pass this in the handler
context = BaseRPCContext(client=client, box=box)  


@box.on_request("/apis/list")
def get_apis() -> list[str]:
    """Respond to a ping request."""
    return {SyftBoxURL.from_path(k, client.workspace): v for k, v in box._SyftEvents__rpc.items()}

@box.on_request("/datasets/create")
def create_dataset(dataset: DatasetCreate) -> Dataset:
    """Respond to a ping request."""
    dataset_service = DatasetService.from_context(context)
    res = dataset_service.create_item(dataset)
    return res
    
@box.on_request("/datasets/list")
def list_datasets() -> list[Dataset]:
    """Respond to a ping request."""
    dataset_service = DatasetService.from_context(context)
    res = dataset_service.list_items()
    return res


2025-02-11 09:50:43.690 | INFO     | syft_event.server2:register_rpc:132 - Register RPC: /apis/list
2025-02-11 09:50:43.691 | INFO     | syft_event.server2:register_rpc:132 - Register RPC: /datasets/create
2025-02-11 09:50:43.693 | INFO     | syft_event.server2:register_rpc:132 - Register RPC: /datasets/list


In [5]:
app_data_dir = client.my_datasite / "apps" / context.box.app_name
import shutil
shutil.rmtree(app_data_dir)

In [6]:
conn = get_connection(box, mock=True)   

In [7]:
# DatasetCreate(name="my-dataset", description="my dataset").to_item()

In [8]:
res = conn.send(
    url=f"syft://{client.email}/api_data/my-rds-app/rpc/apis/list",
    body={},
    expiry="5m",
    cache=True,
)

In [10]:
from syft_rds.api.api import API

In [11]:
api = API.from_email(client.email, conn)

In [12]:
api.datasets.create(body=DatasetCreate(name="my-dataset", description="my dataset"))

full_json [
  {
    "uid": "02e9d884-f1e5-4456-9eae-db6e19d349a0",
    "name": "my-dataset",
    "description": "my dataset"
  }
]


Dataset(uid=UUID('02e9d884-f1e5-4456-9eae-db6e19d349a0'), name='my-dataset', description='my dataset')

In [13]:
api.datasets.list()

[Dataset(uid=UUID('02e9d884-f1e5-4456-9eae-db6e19d349a0'), name='my-dataset', description='my dataset')]

# Appendix

In [14]:
res

{'syft://koen@openmined.org/api_data/my-rds-app/rpc/apis/list': <function __main__.get_apis() -> 'list[str]'>,
 'syft://koen@openmined.org/api_data/my-rds-app/rpc/datasets/create': <function __main__.create_dataset(dataset: 'DatasetCreate') -> 'Dataset'>,
 'syft://koen@openmined.org/api_data/my-rds-app/rpc/datasets/list': <function __main__.list_datasets() -> 'list[Dataset]'>}

In [15]:
res = conn.send(
    url=f"syft://{client.email}/api_data/my-rds-app/rpc/datasets/create",
    body=DatasetCreate(name="my-dataset", description="my dataset"),
    expiry="5m",
    cache=True,
)

full_json [
  {
    "uid": "a0c26121-abbc-4199-a401-57e974fb76df",
    "name": "my-dataset",
    "description": "my dataset"
  },
  {
    "uid": "80774959-e620-42f9-a3c6-5bb020644877",
    "name": "my-dataset",
    "description": "my dataset"
  }
]


In [16]:
res

Dataset(uid=UUID('80774959-e620-42f9-a3c6-5bb020644877'), name='my-dataset', description='my dataset')

: 